# Converter for the Financial Filings at the Sec

In [1]:
from zipfile import ZipFile
import pandas as pd

dirname = "2025q1"
with ZipFile('importFiles/'+dirname+'.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t", low_memory=False)
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

36585510
7507780
224316
815895
24168


In [2]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------------------------------------------------------------------------------------------------------------+----------------------+----------+--------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------------------+------------+
|    | adsh                 | tag                                                                                                           | version              |    ddate |   qtrs | uom    | segments                                                                                                                                                                          | coreg                              |             value |   footnote |
|----+----------------------+-----------------------------------------------------------------------------------

In [3]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------------------------------------------------------------+----------------------+---------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                                                                         | version              | plabel                                                                    |   negating |
|----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------------------------------------------------------------+----------------------+---------------------------------------------------------------------------+------------|
|  0 | 0000002488-25-000012 |        3 |      1 | IS     |       0 | H       | RevenueFromContrac

In [4]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+--------+----------------------------------------+-------+-------------+----------+-----------------+------------+-----------------------------------------+-----------------------------+----------------+-------------+----------+-----------------+------------+-----------------------------------------+------------------------------------+--------------+-----------+-----------+-------------------------------------------+---------------+-------+--------+-------+--------+-------------+------+------+----------+-----------------------+-----------+----------+---------------------------+---------+----------------+
|    | adsh                 |    cik | name                                   |   sic | countryba   | stprba   | cityba          | zipba      | bas1                                    | bas2                        | baph           | countryma   | stprma   | cityma          | zipma      | mas1                                    | mas2                       

In [9]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+------------------------------------------------------------------------------------------------------------------------------------+----------------------+----------+------------+------------+--------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
ddate         int64
qtrs          int64
uom          object
segments     object
coreg        object
value       float64
footnote     object
dtype: object


In [11]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                          0000070145-25-000007
tag         IncreaseDecreaseInAccountsPayableTrade
version                               us-gaap/2024
ddate                                     20241231
qtrs                                             1
uom                                            USD
segments                                       NaN
coreg                                          NaN
value                                    9703000.0
footnote                                       NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'ddate' 'qtrs' 'uom' 'segments' 'coreg' 'value'
 'footnote']


In [12]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [13]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [14]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 611 Dtos size: 291 Bs: 152 Cf: 139 Ic: 0 Json size: 77kb.
file 1 of 6232 exportFiles/2025q1/0000015615-25-000021.json stored in 10901.116ms.
FilterDfNum size: 487 Dtos size: 144 Bs: 86 Cf: 58 Ic: 0 Json size: 38kb.
file 2 of 6232 exportFiles/2025q1/0000016058-25-000032.json stored in 8715.274ms.
FilterDfNum size: 521 Dtos size: 166 Bs: 93 Cf: 73 Ic: 0 Json size: 43kb.
file 3 of 6232 exportFiles/2025q1/0000016732-25-000026.json stored in 9196.76ms.
FilterDfNum size: 455 Dtos size: 182 Bs: 75 Cf: 107 Ic: 0 Json size: 46kb.
file 4 of 6232 exportFiles/2025q1/0000016868-25-000011.json stored in 7811.119ms.
FilterDfNum size: 433 Dtos size: 202 Bs: 113 Cf: 89 Ic: 0 Json size: 52kb.
file 5 of 6232 exportFiles/2025q1/0000016875-25-000008.json stored in 7655.229ms.
FilterDfNum size: 541 Dtos size: 198 Bs: 107 Cf: 91 Ic: 0 Json size: 51kb.
file 6 of 6232 exportFiles/2025q1/0000016918-25-000004.json stored in 9992.9ms.
FilterDfNum size: 5183 Dtos size: 157 Bs: 83 Cf: 74 Ic: 0 Jso

FilterDfNum size: 465 Dtos size: 148 Bs: 86 Cf: 62 Ic: 0 Json size: 35kb.
file 62 of 6492 exportFiles/2024q4/0001437749-24-035078.json stored in 8440.848ms.
FilterDfNum size: 274 Dtos size: 116 Bs: 75 Cf: 41 Ic: 0 Json size: 26kb.
file 63 of 6492 exportFiles/2024q4/0001213900-24-100247.json stored in 5031.218ms.
FilterDfNum size: 224 Dtos size: 82 Bs: 48 Cf: 34 Ic: 0 Json size: 19kb.
file 64 of 6492 exportFiles/2024q4/0001477932-24-006974.json stored in 4173.542ms.
FilterDfNum size: 276 Dtos size: 143 Bs: 79 Cf: 64 Ic: 0 Json size: 32kb.
file 65 of 6492 exportFiles/2024q4/0001654954-24-014179.json stored in 5137.549ms.
FilterDfNum size: 402 Dtos size: 153 Bs: 96 Cf: 57 Ic: 0 Json size: 54kb.
file 66 of 6492 exportFiles/2024q4/0001558370-24-015711.json stored in 7094.287ms.
FilterDfNum size: 432 Dtos size: 141 Bs: 93 Cf: 48 Ic: 0 Json size: 34kb.
file 67 of 6492 exportFiles/2024q4/0001493152-24-044751.json stored in 7670.662ms.
FilterDfNum size: 505 Dtos size: 274 Bs: 226 Cf: 48 Ic: 0 J

FilterDfNum size: 1103 Dtos size: 354 Bs: 294 Cf: 60 Ic: 0 Json size: 88kb.
file 121 of 6492 exportFiles/2024q4/0000036104-24-000072.json stored in 19493.415ms.
FilterDfNum size: 456 Dtos size: 171 Bs: 98 Cf: 73 Ic: 0 Json size: 42kb.
file 122 of 6492 exportFiles/2024q4/0001632127-24-000125.json stored in 8231.121ms.
FilterDfNum size: 348 Dtos size: 191 Bs: 121 Cf: 70 Ic: 0 Json size: 65kb.
file 123 of 6492 exportFiles/2024q4/0000950170-24-115082.json stored in 6157.377ms.
FilterDfNum size: 482 Dtos size: 237 Bs: 165 Cf: 72 Ic: 0 Json size: 58kb.
file 124 of 6492 exportFiles/2024q4/0000798354-24-000190.json stored in 8776.269ms.
FilterDfNum size: 146 Dtos size: 81 Bs: 59 Cf: 22 Ic: 0 Json size: 19kb.
file 125 of 6492 exportFiles/2024q4/0001096906-24-002215.json stored in 2841.388ms.
FilterDfNum size: 469 Dtos size: 156 Bs: 94 Cf: 62 Ic: 0 Json size: 37kb.
file 126 of 6492 exportFiles/2024q4/0000746838-24-000030.json stored in 8341.836ms.
FilterDfNum size: 1074 Dtos size: 436 Bs: 361 Cf

FilterDfNum size: 885 Dtos size: 392 Bs: 303 Cf: 89 Ic: 0 Json size: 120kb.
file 181 of 6492 exportFiles/2024q4/0001437958-24-000161.json stored in 16311.311ms.
FilterDfNum size: 377 Dtos size: 142 Bs: 68 Cf: 74 Ic: 0 Json size: 34kb.
file 182 of 6492 exportFiles/2024q4/0001326801-24-000081.json stored in 7157.632ms.
FilterDfNum size: 394 Dtos size: 138 Bs: 70 Cf: 68 Ic: 0 Json size: 35kb.
file 183 of 6492 exportFiles/2024q4/0001820144-24-000054.json stored in 7410.04ms.
FilterDfNum size: 596 Dtos size: 141 Bs: 67 Cf: 74 Ic: 0 Json size: 34kb.
file 184 of 6492 exportFiles/2024q4/0000096943-24-000148.json stored in 11435.224ms.
FilterDfNum size: 831 Dtos size: 263 Bs: 166 Cf: 97 Ic: 0 Json size: 63kb.
file 185 of 6492 exportFiles/2024q4/0001061219-24-000021.json stored in 15579.577ms.
FilterDfNum size: 340 Dtos size: 144 Bs: 72 Cf: 72 Ic: 0 Json size: 33kb.
file 186 of 6492 exportFiles/2024q4/0001477932-24-007062.json stored in 6563.312ms.
FilterDfNum size: 996 Dtos size: 553 Bs: 471 Cf

FilterDfNum size: 513 Dtos size: 216 Bs: 169 Cf: 47 Ic: 0 Json size: 53kb.
file 235 of 6492 exportFiles/2024q4/0000732712-24-000069.json stored in 9667.339ms.
FilterDfNum size: 432 Dtos size: 140 Bs: 96 Cf: 44 Ic: 0 Json size: 39kb.
file 236 of 6492 exportFiles/2024q4/0001410578-24-001983.json stored in 8508.865ms.
File exportFiles/2024q4/0001493152-24-042875.json has Symbol Series([], )
FilterDfNum size: 799 Dtos size: 342 Bs: 228 Cf: 114 Ic: 0 Json size: 90kb.
file 238 of 6492 exportFiles/2024q4/0000950170-24-125133.json stored in 15214.292ms.
FilterDfNum size: 453 Dtos size: 175 Bs: 116 Cf: 59 Ic: 0 Json size: 45kb.
file 239 of 6492 exportFiles/2024q4/0001477932-24-008271.json stored in 8762.636ms.
FilterDfNum size: 245 Dtos size: 92 Bs: 64 Cf: 28 Ic: 0 Json size: 28kb.
file 240 of 6492 exportFiles/2024q4/0001493152-24-045892.json stored in 5188.288ms.
FilterDfNum size: 400 Dtos size: 121 Bs: 53 Cf: 68 Ic: 0 Json size: 29kb.
file 241 of 6492 exportFiles/2024q4/0001437749-24-033608.j

...

FilterDfNum size: 667 Dtos size: 290 Bs: 216 Cf: 74 Ic: 0 Json size: 66kb.
file 6238 of 6492 exportFiles/2024q4/0001683168-24-007656.json stored in 11541.054ms.
FilterDfNum size: 257 Dtos size: 125 Bs: 74 Cf: 51 Ic: 0 Json size: 31kb.
file 6239 of 6492 exportFiles/2024q4/0001493152-24-048103.json stored in 4646.625ms.
FilterDfNum size: 564 Dtos size: 175 Bs: 87 Cf: 88 Ic: 0 Json size: 41kb.
file 6240 of 6492 exportFiles/2024q4/0000950170-24-122768.json stored in 9829.516ms.
FilterDfNum size: 284 Dtos size: 142 Bs: 85 Cf: 57 Ic: 0 Json size: 34kb.
file 6241 of 6492 exportFiles/2024q4/0001636282-24-000109.json stored in 5120.291ms.
FilterDfNum size: 251 Dtos size: 71 Bs: 44 Cf: 27 Ic: 0 Json size: 17kb.
file 6242 of 6492 exportFiles/2024q4/0001410578-24-001791.json stored in 4572.908ms.
FilterDfNum size: 272 Dtos size: 102 Bs: 65 Cf: 37 Ic: 0 Json size: 29kb.
file 6243 of 6492 exportFiles/2024q4/0001477932-24-007285.json stored in 4956.786ms.
FilterDfNum size: 463 Dtos size: 251 Bs: 112 

FilterDfNum size: 500 Dtos size: 198 Bs: 113 Cf: 85 Ic: 0 Json size: 44kb.
file 6296 of 6492 exportFiles/2024q4/0001477932-24-007237.json stored in 8896.475ms.
FilterDfNum size: 479 Dtos size: 167 Bs: 110 Cf: 57 Ic: 0 Json size: 41kb.
file 6297 of 6492 exportFiles/2024q4/0001140361-24-049040.json stored in 8445.637ms.
FilterDfNum size: 933 Dtos size: 454 Bs: 274 Cf: 180 Ic: 0 Json size: 109kb.
file 6298 of 6492 exportFiles/2024q4/0001213900-24-111385.json stored in 16002.591ms.
FilterDfNum size: 475 Dtos size: 138 Bs: 82 Cf: 56 Ic: 0 Json size: 32kb.
file 6299 of 6492 exportFiles/2024q4/0001602065-24-000084.json stored in 8340.608ms.
FilterDfNum size: 396 Dtos size: 166 Bs: 112 Cf: 54 Ic: 0 Json size: 42kb.
file 6300 of 6492 exportFiles/2024q4/0001878897-24-000055.json stored in 7055.231ms.
FilterDfNum size: 107 Dtos size: 56 Bs: 38 Cf: 18 Ic: 0 Json size: 19kb.
file 6302 of 6492 exportFiles/2024q4/0001640334-24-001476.json stored in 2225.238ms.
FilterDfNum size: 714 Dtos size: 206 Bs:

FilterDfNum size: 605 Dtos size: 160 Bs: 96 Cf: 64 Ic: 0 Json size: 39kb.
file 6357 of 6492 exportFiles/2024q4/0001437749-24-037110.json stored in 10703.509ms.
FilterDfNum size: 347 Dtos size: 131 Bs: 57 Cf: 74 Ic: 0 Json size: 37kb.
file 6359 of 6492 exportFiles/2024q4/0000950170-24-126296.json stored in 6195.568ms.
FilterDfNum size: 265 Dtos size: 111 Bs: 78 Cf: 33 Ic: 0 Json size: 26kb.
file 6360 of 6492 exportFiles/2024q4/0001213900-24-087502.json stored in 4812.009ms.
FilterDfNum size: 499 Dtos size: 161 Bs: 86 Cf: 75 Ic: 0 Json size: 38kb.
file 6362 of 6492 exportFiles/2024q4/0001437749-24-034916.json stored in 8759.919ms.
FilterDfNum size: 609 Dtos size: 223 Bs: 145 Cf: 78 Ic: 0 Json size: 52kb.
file 6363 of 6492 exportFiles/2024q4/0000950170-24-119420.json stored in 10654.855ms.
FilterDfNum size: 506 Dtos size: 154 Bs: 81 Cf: 73 Ic: 0 Json size: 36kb.
file 6364 of 6492 exportFiles/2024q4/0001558370-24-013452.json stored in 8980.237ms.
FilterDfNum size: 206 Dtos size: 138 Bs: 10

FilterDfNum size: 386 Dtos size: 156 Bs: 106 Cf: 50 Ic: 0 Json size: 37kb.
file 6418 of 6492 exportFiles/2024q4/0001410578-24-001694.json stored in 6938.459ms.
FilterDfNum size: 190 Dtos size: 87 Bs: 56 Cf: 31 Ic: 0 Json size: 29kb.
file 6419 of 6492 exportFiles/2024q4/0001829126-24-006659.json stored in 3583.085ms.
FilterDfNum size: 304 Dtos size: 115 Bs: 71 Cf: 44 Ic: 0 Json size: 38kb.
file 6420 of 6492 exportFiles/2024q4/0001108205-24-000072.json stored in 5543.281ms.
FilterDfNum size: 155 Dtos size: 46 Bs: 20 Cf: 26 Ic: 0 Json size: 11kb.
file 6421 of 6492 exportFiles/2024q4/0000950170-24-122614.json stored in 2972.871ms.
FilterDfNum size: 546 Dtos size: 301 Bs: 186 Cf: 115 Ic: 0 Json size: 70kb.
file 6422 of 6492 exportFiles/2024q4/0000010795-24-000084.json stored in 9706.752ms.
FilterDfNum size: 437 Dtos size: 138 Bs: 81 Cf: 57 Ic: 0 Json size: 35kb.
file 6423 of 6492 exportFiles/2024q4/0000096869-24-000014.json stored in 7777.208ms.
FilterDfNum size: 1231 Dtos size: 241 Bs: 135

FilterDfNum size: 409 Dtos size: 139 Bs: 76 Cf: 63 Ic: 0 Json size: 33kb.
file 6475 of 6492 exportFiles/2024q4/0001628280-24-044572.json stored in 7380.258ms.
FilterDfNum size: 325 Dtos size: 108 Bs: 66 Cf: 42 Ic: 0 Json size: 25kb.
file 6476 of 6492 exportFiles/2024q4/0001185185-24-001122.json stored in 5904.613ms.
FilterDfNum size: 472 Dtos size: 130 Bs: 80 Cf: 50 Ic: 0 Json size: 31kb.
file 6477 of 6492 exportFiles/2024q4/0001437749-24-032782.json stored in 8288.638ms.
FilterDfNum size: 642 Dtos size: 169 Bs: 91 Cf: 78 Ic: 0 Json size: 41kb.
file 6478 of 6492 exportFiles/2024q4/0001777835-24-000187.json stored in 11210.652ms.
FilterDfNum size: 254 Dtos size: 154 Bs: 85 Cf: 69 Ic: 0 Json size: 35kb.
file 6479 of 6492 exportFiles/2024q4/0001683168-24-008114.json stored in 4620.419ms.
FilterDfNum size: 568 Dtos size: 244 Bs: 123 Cf: 121 Ic: 0 Json size: 71kb.
file 6480 of 6492 exportFiles/2024q4/0000707388-24-000181.json stored in 9911.364ms.
FilterDfNum size: 166 Dtos size: 70 Bs: 47 

In [19]:
import pathlib
import os
from zipfile import ZipFile, ZIP_DEFLATED

if os.path.exists("export.zip"):
    os.remove("export.zip")

directory = pathlib.Path("exportFiles/")
with ZipFile("export.zip", mode="w", compression=ZIP_DEFLATED, compresslevel=9) as archive:
    for file_path in directory.rglob("*"):
      archive.write(file_path, arcname=file_path.relative_to(directory))